In [ ]:
#author:WitsonTweets
#This Module is a LSTM Neural Network to solve the classification Problem 

In [1]:
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils import plot_model
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import re

Using TensorFlow backend.


In [2]:
from keras.preprocessing.sequence import pad_sequences
training =pd.read_csv('2lakh1.csv',usecols=(0,1),encoding="latin-1")
stop = set(stopwords.words('english'))
train_x=training.iloc[:,1].astype(str)
train_y=training.iloc[:,0]

# maximum number of words used for training the dataset
max_words = 3000

#Create a tokenizer using Keras and input the tweets
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)

dictionary = tokenizer.word_index

#Saving the dictionary to a json file so we can use it for processing Tweets while testing the model

with open('dictionarylstm.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)
    
    
train_x = tokenizer.texts_to_sequences(train_x)
train_x = pad_sequences(train_x)
#print(train_x[0:5])

train_y = keras.utils.to_categorical(train_y, 2)




In [3]:
print(len(train_x[0]))

25


In [4]:
embedding_size=128
lstm_out=196

from keras.callbacks import EarlyStopping

model=Sequential()
model.add(Embedding(max_words,embedding_size,input_length=len(train_x[0])))
model.add(LSTM(lstm_out,recurrent_dropout=0.5,dropout=0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_acc',mode='max')
print(model.summary())
#plot_model(model,to_file="lstmmodel.png",show_shapes=True,show_layer_names=True)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 128)           384000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 639,194
Trainable params: 639,194
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit(train_x, train_y,
    batch_size=28,
    epochs=7,
    verbose=1,
    validation_split=0.1,shuffle=True)

Train on 177592 samples, validate on 19733 samples
Epoch 1/7
177592/177592 [==============================] - 617s 3ms/step - loss: 0.5356 - acc: 0.7261 - val_loss: 0.4838 - val_acc: 0.7603
Epoch 2/7
177592/177592 [==============================] - 603s 3ms/step - loss: 0.5065 - acc: 0.7479 - val_loss: 0.4793 - val_acc: 0.7627
Epoch 3/7
177592/177592 [==============================] - 611s 3ms/step - loss: 0.4954 - acc: 0.7553 - val_loss: 0.4857 - val_acc: 0.7586
Epoch 4/7
177592/177592 [==============================] - 615s 3ms/step - loss: 0.4864 - acc: 0.7609 - val_loss: 0.4833 - val_acc: 0.7630
Epoch 5/7
177592/177592 [==============================] - 612s 3ms/step - loss: 0.4794 - acc: 0.7643 - val_loss: 0.4832 - val_acc: 0.7576
Epoch 6/7
177592/177592 [==============================] - 614s 3ms/step - loss: 0.4724 - acc: 0.7698 - val_loss: 0.4980 - val_acc: 0.7546
Epoch 7/7
177592/177592 [==============================] - 640s 4ms/step - loss: 0.4664 - acc: 0.7732 - val_loss: 0

In [6]:
model_json = model.to_json()
with open('modellstm.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('modellstm.h5')

print('saved model!')

saved model!
